In [66]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plp 
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [67]:
df = pd.read_csv('../data/VOO.csv')

In [68]:
df.head(3)

,date,close,high,low,open,volume,daily_return,rsi,macd,macd_signal,...,ema_200,ema_9,ichimoku_a,ichimoku_b,ichimoku_base,ichimoku_conversion,adx,volume_sma_20,volume_ratio,awesome_osc
0,2011-06-23,91.127,91.158,89.689,90.338,260750,-0.003,44.614,-0.773,-0.887,...,89.286,91.098,91.583,93.16,92.395,90.771,21.808,116412.5,2.240,-2.011
1,2011-06-24,90.125,91.181,89.969,91.150,114500,-0.011,39.853,-0.802,-0.870,...,89.295,90.904,91.583,93.16,92.395,90.771,21.984,119115.0,0.961,-1.842
2,2011-06-27,90.886,91.228,89.985,90.187,77850,0.008,44.681,-0.756,-0.847,...,89.310,90.900,91.564,93.16,92.356,90.771,22.095,120627.5,0.645,-1.670


In [69]:
df['date'] = pd.to_datetime(df['date'])

### SEÑALES

In [70]:
# -----------------------------------------------------------------------------
# ESTRATEGIAS DE SEÑALES DE COMPRA
# -----------------------------------------------------------------------------

## 1. Estrategia: Cruce de Precio sobre Media Móvil (Tendencia) 
def ema_price_signal(df, ema_fast='ema_20', close='close', rsi='rsi'):
    df['ema_price_diff'] = df[close] - df[ema_fast]

    df['signal_ema_price'] = (
        (df['ema_price_diff'] > 0) 
        & (df['ema_price_diff'].shift(1) <= 0) 
        & (df[close] > df[ema_fast]) 
        & (df[close].shift(1) <= df[ema_fast].shift(1)) 
        & (df[rsi].between(20, 50))
        & (df[rsi].shift(1).between(20, 50))
    )

    return df

## 2. Estrategia: Cruce de MACD (Momentum) 
def macd_signal(df, macd_line='macd', signal_line='macd_signal', ema_long='ema_50', close='close'):
    df['macd_diff'] = df[macd_line] - df[signal_line]
    df['signal_macd_buy'] = (
        (df['macd_diff'] > 0)
        & (df['macd_diff'].shift(1) <= 0)
        & (df[macd_line] < 0)
        & (df[close] > df[ema_long])
    )
    return df

## 3. Estrategia: Oscilador Estocástico (Reversión) 
def stochastic_oversold_signal(df, k_line='stochastic_k', d_line='stochastic_d', ema_long='ema_50', close='close'):
    is_oversold = (df[k_line].shift(1) < 20) & (df[d_line].shift(1) < 20)
    k_crosses_d_up = (df[k_line] > df[d_line]) & (df[k_line].shift(1) <= df[d_line].shift(1))
    is_uptrend = (df[close] > df[ema_long])
    df['signal_stochastic_buy'] = is_oversold & k_crosses_d_up & is_uptrend
    return df

## 4. Estrategias con Ichimoku (Equilibrio y Soportes) 

# 4.1 Ichimoku Conservadora 
def ichimoku_buy_signals(df, tenkan='ichimoku_conversion', kijun='ichimoku_base',senkou_a='ichimoku_a', senkou_b='ichimoku_b',close='close', rsi='rsi', adx='adx'):
    df['signal_ichimoku_buy'] = (
        (df[close] > df[senkou_a])
        & (df[close] > df[senkou_b])
        & (df[tenkan] > df[kijun])
        & (df[tenkan].shift(1) <= df[kijun].shift(1))
        & (df[rsi] < 70)
        & (df[adx] > 20)
    )
    return df

# 4.2 Ichimoku Agresiva (Cruce dentro de la nube)
def ichimoku_signal_aggressive(df, tenkan='ichimoku_conversion', kijun='ichimoku_base',senkou_a='ichimoku_a', senkou_b='ichimoku_b',
                               close='close', adx='adx'):
    inside_cloud = (
        (df[close] > df[[senkou_a, senkou_b]].min(axis=1)) &
        (df[close] < df[[senkou_a, senkou_b]].max(axis=1))
    )
    df['signal_ichimoku_aggressive'] = (
        (df[tenkan] > df[kijun])
        & (df[tenkan].shift(1) <= df[kijun].shift(1))
        & inside_cloud
        & (df[adx] > 18)
    )
    return df

# 4.3 Ichimoku Cruce del Kijun (Señal temprana de equilibrio)
def ichimoku_signal_kijun_cross(df, kijun='ichimoku_base', senkou_a='ichimoku_a',close='close', rsi='rsi'):
    df['signal_ichimoku_kijun_cross'] = (
        (df[close] > df[kijun])
        & (df[close].shift(1) <= df[kijun].shift(1))
        & (df[close] > df[senkou_a])
        & (df[rsi].between(40, 60))
    )
    return df

# 4.4 Ichimoku Ruptura del Chikou Span (Confirmación rápida)
def ichimoku_signal_chikou_break(df, tenkan='ichimoku_conversion', kijun='ichimoku_base',close='close'):
    chikou_span = df[close].shift(-26)
    past_price = df[close]
    df['signal_ichimoku_chikou'] = (
        (chikou_span > past_price)
        & (chikou_span.shift(1) <= past_price.shift(1))
        & (df[close] > df[tenkan])
        & (df[tenkan] > df[kijun])
    )
    return df

def bollinger_reversion_signal(df, close_col='close', lband_col='bollinger_lband',ema_long='ema_20', rsi_col='rsi'):
    """
    Genera una señal de compra por reversión a la media usando Bandas de Bollinger.
    """
    #El precio de ayer estaba por encima, y hoy está por debajo de la banda inferior.
    price_crosses_lband = (df[close_col] < df[lband_col]) & (df[close_col].shift(1) >= df[lband_col].shift(1))
    
    #  La tendencia principal es alcista
    is_uptrend = df[close_col] > df[ema_long]
    
    # El activo está en sobreventa según el RSI
    is_oversold = df[rsi_col] < 30
    
    df['signal_bollinger_buy'] = price_crosses_lband & is_uptrend & is_oversold
    
    return df


In [71]:
#Llamar a las funciones para agregar las señales al DataFrame
df = ema_price_signal(df)
df = macd_signal(df)
df = stochastic_oversold_signal(df)
df = ichimoku_buy_signals(df)
df = ichimoku_signal_aggressive(df)
df = ichimoku_signal_kijun_cross(df)
df = ichimoku_signal_chikou_break(df)
df = bollinger_reversion_signal(df)


In [72]:
def create_combined_signal(df, signal1, signal2, window=3):
    """
    Genera una nueva columna en el DataFrame para una señal combinada.
    La nueva columna se nombra dinámicamente, ej: 'signal_macd_&_stochastic'.
    
    Devuelve el DataFrame modificado y el nombre de la nueva señal creada.
    """
    
    name1 = signal1.replace('signal_', '').replace('_buy', '')
    name2 = signal2.replace('signal_', '').replace('_buy', '')
    new_signal_name = f"signal_{name1}_&_{name2}"
    
    
    if signal1 not in df.columns or signal2 not in df.columns:
        print(f"Error: No se encontraron las columnas de señales '{signal1}' o '{signal2}'.")
        return df, None

    
    confirmation_in_window = df[signal2].rolling(window=window, min_periods=1).sum() > 0
    df[new_signal_name] = (df[signal1] == True) & (confirmation_in_window == True)
    
    print(f"Señal combinada '{new_signal_name}' creada.")
    
    return df, new_signal_name

In [73]:
def generar_estrategias_combinadas(df: pd.DataFrame, combinaciones_a_crear: list, window: int = 3) -> tuple[pd.DataFrame, list]:
    """
    Crea múltiples columnas de señales combinadas en un DataFrame a partir de una lista de pares.

    Args:
        df (pd.DataFrame): El DataFrame que ya contiene las señales individuales.
        combinaciones_a_crear (list): Una lista de tuplas, donde cada tupla contiene dos nombres de señales a combinar (principal, confirmación).
        window (int): La ventana de días para la señal de confirmación.

    Returns:
        tuple[pd.DataFrame, list]: Una tupla conteniendo el DataFrame modificado y una lista con los nombres de las nuevas columnas de señales creadas.
    """
    estrategias_combinadas_creadas = []
    
    print(" Creando Estrategias Combinadas")
    
    
    for signal_principal, signal_confirmacion in combinaciones_a_crear:
        
        
        df, nueva_senal = create_combined_signal(
            df,
            signal1=signal_principal,
            signal2=signal_confirmacion,
            window=window
        )
        
        
        if nueva_senal:
            estrategias_combinadas_creadas.append(nueva_senal)

    print("Se han añadido las siguientes columnas de señales combinadas al dataframe:")
    for nombre in estrategias_combinadas_creadas:
        print(f"- {nombre}")
        
    return df, estrategias_combinadas_creadas


In [74]:
combinaciones = [
    ('signal_macd_buy', 'signal_stochastic_buy'),
    ('signal_ema_price', 'signal_macd_buy'),
    ('signal_ichimoku_kijun_cross', 'signal_stochastic_buy'),
    ('signal_ema_price', 'signal_stochastic_buy')]

df, nuevas_estrategias = generar_estrategias_combinadas(df, combinaciones)

 Creando Estrategias Combinadas
Señal combinada 'signal_macd_&_stochastic' creada.
Señal combinada 'signal_ema_price_&_macd' creada.
Señal combinada 'signal_ichimoku_kijun_cross_&_stochastic' creada.
Señal combinada 'signal_ema_price_&_stochastic' creada.
Se han añadido las siguientes columnas de señales combinadas al dataframe:
- signal_macd_&_stochastic
- signal_ema_price_&_macd
- signal_ichimoku_kijun_cross_&_stochastic
- signal_ema_price_&_stochastic


### Backtest

In [75]:
def backtest_dca_pure(df, price_col='close', date_col='date', monthly_invest=200):
    """
    Calcula el rendimiento de una estrategia de inversión pura de
    Aportaciones Periódicas (Dollar-Cost Averaging).
    """
    #  Asegura de que la columna de fecha es de tipo datetime ---
    df[date_col] = pd.to_datetime(df[date_col])
    
    
    dca_shares = 0
    total_contributions = 0
    last_month = None
    portfolio_values = []
    contributions_history = []

    
    for i, row in df.iterrows():
        current_price = row[price_col]
        current_date = row[date_col]
        
        # logica para la aportacion mensual
        if last_month is None or current_date.month != last_month:
            dca_shares += monthly_invest / current_price
            total_contributions += monthly_invest
            last_month = current_date.month
        
        portfolio_values.append(dca_shares * current_price)
        contributions_history.append(total_contributions)
            
    final_value = portfolio_values[-1]
    final_contributions = contributions_history[-1]
    absolute_return = final_value - final_contributions
    percentage_return = (absolute_return / final_contributions) * 100 if final_contributions > 0 else 0

    results = {
        'Strategy': 'DCA Puro (Benchmark)',
        'Final Portfolio Value': final_value,
        'Total Contributions': final_contributions,
        'Absolute Return': absolute_return,
        'Percentage Return': percentage_return,
        'Trading PnL': 0,
        'Trading Capital Used': 0,
        'Open Trades at End': 0,
        'Value of Open Trades': 0
    }
    
    print(" Resultados para la estrategia: DCA Puro ")
    print(f"Valor Final del Portafolio: ${final_value:,.2f}")
    print(f"Total Aportado: ${final_contributions:,.2f}")
    print(f"Retorno Porcentual: {percentage_return:.2f}%\n")

    return results, df.assign(portfolio_value=portfolio_values, contributions=contributions_history)

In [76]:
def backtest_dca_plus_trading_WITH_SL(df, price_col='close', 
                                    signal_col='signal_ema_price',monthly_invest=200, 
                                    trade_amount=50,take_profit_pct=0.10, stop_loss_pct=-0.05, 
                                    date_col='date', initial_trading_cash=1000, 
                                    verbose=True):
    """
    Ejecuta un backtest CON Stop-Loss y reporta las posiciones que quedan abiertas al final.
    """
    df[date_col] = pd.to_datetime(df[date_col])
    
    cash = initial_trading_cash
    dca_shares = 0
    open_trades = []
    total_dca_contributions = 0
    trading_contributions = 0
    trading_pnl = 0
    last_month = None

    for i, row in df.iterrows():
        current_price = row[price_col]
        current_date = row[date_col]
        
        if last_month is None or current_date.month != last_month:
            dca_shares += monthly_invest / current_price
            total_dca_contributions += monthly_invest
            last_month = current_date.month
            
        remaining_trades = []
        for trade in open_trades:
            return_pct = (current_price - trade['buy_price']) / trade['buy_price']
            if return_pct >= take_profit_pct or return_pct <= stop_loss_pct:
                sell_value = trade['shares'] * current_price
                cash += sell_value
                profit_or_loss = sell_value - (trade['shares'] * trade['buy_price'])
                trading_pnl += profit_or_loss
            else:
                remaining_trades.append(trade)
        open_trades = remaining_trades

        if row[signal_col] and cash >= trade_amount:
            shares_bought = trade_amount / current_price
            cash -= trade_amount
            trading_contributions += trade_amount
            open_trades.append({'buy_price': current_price, 'shares': shares_bought})
            
    
    final_price = df[price_col].iloc[-1]
    dca_value = dca_shares * final_price
    
    
    open_trades_at_end = len(open_trades)
    value_of_open_trades = sum(trade['shares'] * final_price for trade in open_trades)
    
    final_value = cash + dca_value + value_of_open_trades
    final_contributions = total_dca_contributions + initial_trading_cash
    absolute_return = final_value - final_contributions
    percentage_return = (absolute_return / final_contributions) * 100 if final_contributions > 0 else 0

    results = {
        'Strategy': f"{signal_col} (With SL)",
        'Final Portfolio Value': final_value,
        'Total Contributions': final_contributions,
        'Absolute Return': absolute_return,
        'Percentage Return': percentage_return,
        'Trading PnL': trading_pnl,
        'Trading Capital Used': trading_contributions,
        'Open Trades at End': open_trades_at_end,
        'Value of Open Trades': value_of_open_trades
    }
    
    if verbose:
        print(f"Resultados para: {signal_col} (Con SL)")
        print(f"Retorno Porcentual: {percentage_return:.2f}%")
        print(f"Posiciones Abiertas al Final: {open_trades_at_end}")
        print(f"Valor de Posiciones Abiertas: ${value_of_open_trades:,.2f}\n")

    return results

In [77]:
def backtest_with_atr_SL(df, price_col='close', signal_col='signal_ema_price',
                        monthly_invest=200, trade_amount=50,
                        atr_multiplier_tp=3.0,  # Take profit a 3 veces el ATR
                        atr_multiplier_sl=1.5,  # Stop loss a 1.5 veces el ATR
                        date_col='date', initial_trading_cash=1000, atr_col='atr', verbose=True):
    """
    Ejecuta un backtest donde el Take Profit y Stop Loss tomando el ATR como multiplicador haciendo que ambos sean dinamicos.
    """
    df[date_col] = pd.to_datetime(df[date_col])
    cash, dca_shares, open_trades, total_dca_contributions, trading_contributions, trading_pnl, last_month = initial_trading_cash, 0, [], 0, 0, 0, None
    
    for i, row in df.iterrows():
        current_price, current_date = row[price_col], row[date_col]
        
        if last_month is None or current_date.month != last_month:
            dca_shares += monthly_invest / current_price
            total_dca_contributions += monthly_invest
            last_month = current_date.month
            
        remaining_trades = []
        for trade in open_trades:
            # Venda dinamica
            if current_price >= trade['take_profit_price'] or current_price <= trade['stop_loss_price']:
                sell_value = trade['shares'] * current_price
                cash += sell_value
                profit_or_loss = sell_value - (trade['shares'] * trade['buy_price'])
                trading_pnl += profit_or_loss
            else:
                remaining_trades.append(trade)
        open_trades = remaining_trades

        if row[signal_col] and cash >= trade_amount:
            shares_bought = trade_amount / current_price
            cash -= trade_amount
            trading_contributions += trade_amount
            
            # Compra con stops dinamicos
            atr_at_buy = row[atr_col]
            stop_loss_price = current_price - (atr_multiplier_sl * atr_at_buy)
            take_profit_price = current_price + (atr_multiplier_tp * atr_at_buy)
            
            open_trades.append({
                'buy_price': current_price, 
                'shares': shares_bought,
                'stop_loss_price': stop_loss_price,
                'take_profit_price': take_profit_price
            })
            
    
    final_value = cash + (dca_shares * df[price_col].iloc[-1]) + sum(t['shares'] * df[price_col].iloc[-1] for t in open_trades)
    final_contributions = total_dca_contributions + initial_trading_cash
    absolute_return = final_value - final_contributions
    percentage_return = (absolute_return / final_contributions) * 100 if final_contributions > 0 else 0
    open_trades_at_end = len(open_trades)
    value_of_open_trades = sum(trade['shares'] * df[price_col].iloc[-1] for trade in open_trades)
    
    return {'Strategy': f"{signal_col} (ATR Stops)", 'Final Portfolio Value': final_value, 
            'Total Contributions': final_contributions, 'Absolute Return': absolute_return, 
            'Percentage Return': percentage_return, 'Trading PnL': trading_pnl, 
            'Trading Capital Used': trading_contributions, 'Open Trades at End': open_trades_at_end, 
            'Value of Open Trades': value_of_open_trades}

In [78]:
def backtest_dca_plus_trading_NO_SL(df, price_col='close', signal_col='signal_ema_price',
                                    monthly_invest=200, trade_amount=50,
                                    take_profit_pct=0.10,
                                    date_col='date', initial_trading_cash=1000, verbose=True):
    """
    Ejecuta un backtest sin Stop-Loss indicando las posiciones que quedan abiertas en la estrategia.
    """
    df[date_col] = pd.to_datetime(df[date_col])
    
    cash = initial_trading_cash
    dca_shares = 0
    open_trades = []
    total_dca_contributions = 0
    trading_contributions = 0
    trading_pnl = 0
    last_month = None

    for i, row in df.iterrows():
        current_price = row[price_col]
        current_date = row[date_col]
        
        if last_month is None or current_date.month != last_month:
            dca_shares += monthly_invest / current_price
            total_dca_contributions += monthly_invest
            last_month = current_date.month
            
        remaining_trades = []
        for trade in open_trades:
            return_pct = (current_price - trade['buy_price']) / trade['buy_price']
            if return_pct >= take_profit_pct:
                sell_value = trade['shares'] * current_price
                cash += sell_value
                profit = sell_value - (trade['shares'] * trade['buy_price'])
                trading_pnl += profit
            else:
                remaining_trades.append(trade)
        open_trades = remaining_trades

        if row[signal_col] and cash >= trade_amount:
            shares_bought = trade_amount / current_price
            cash -= trade_amount
            trading_contributions += trade_amount
            open_trades.append({'buy_price': current_price, 'shares': shares_bought})
            

    final_price = df[price_col].iloc[-1]
    dca_value = dca_shares * final_price
    

    open_trades_at_end = len(open_trades)
    value_of_open_trades = sum(trade['shares'] * final_price for trade in open_trades)
    
    final_value = cash + dca_value + value_of_open_trades
    final_contributions = total_dca_contributions + initial_trading_cash
    absolute_return = final_value - final_contributions
    percentage_return = (absolute_return / final_contributions) * 100 if final_contributions > 0 else 0

    results = {
        'Strategy': f"{signal_col} (No SL)",
        'Final Portfolio Value': final_value,
        'Total Contributions': final_contributions,
        'Absolute Return': absolute_return,
        'Percentage Return': percentage_return,
        'Trading PnL': trading_pnl,
        'Trading Capital Used': trading_contributions,
        'Open Trades at End': open_trades_at_end,    
        'Value of Open Trades': value_of_open_trades
    }
    
    if verbose:
        print(f"--- Resultados para: {signal_col} (Sin SL) ---")
        print(f"Retorno Porcentual: {percentage_return:.2f}%")
        print(f"Posiciones Abiertas al Final: {open_trades_at_end}")
        print(f"Valor de Posiciones Abiertas: ${value_of_open_trades:,.2f}\n")

    return results

In [79]:

def evaluate_all_strategies(df_original, signal_columns, backtest_func, backtest_params):
    """
    Ejecuta el backtest para una lista de estrategias y devuelve un DataFrame con los resultados.
    
    Args:
        df_original (pd.DataFrame): Tu DataFrame con todos los indicadores calculados.
        signal_columns (list): Una lista con los nombres de las columnas de señales a probar.
        backtest_func (function): La función de backtest a utilizar (ej. backtest_with_sl).
        backtest_params (dict): Un diccionario con los parámetros para el backtest.
        
    Returns:
        pd.DataFrame: Un nuevo DataFrame con el resumen de rendimiento de cada estrategia.
    """
    results_list = []
    
    for signal in signal_columns:
        print(f"Evaluando: {signal}...")
        df_copy = df_original.copy()
        
        results = backtest_func(
            df_copy,
            signal_col=signal,
            **backtest_params
        )
        results_list.append(results)
        
    results_df = pd.DataFrame(results_list).sort_values(by='Percentage Return', ascending=False).reset_index(drop=True)
    
    return results_df

In [80]:
#Parametros para los backtests 

estrategias_a_probar = [
    # Individuales
    'signal_ema_price',
    'signal_macd_buy',
    'signal_stochastic_buy',
    'signal_ichimoku_kijun_cross',
    'signal_bollinger_buy',
    
    # Combinadas
    'signal_macd_&_stochastic',
    'signal_ema_price_&_macd',
    'signal_ichimoku_kijun_cross_&_stochastic',
    'signal_ema_price_&_stochastic'    
]

params = {
    'monthly_invest': 400,
    'trade_amount': 150,
    'take_profit_pct': 0.125,
    'initial_trading_cash': 1000,
    'verbose': False 
}

params_atr = {
    'monthly_invest': 400,
    'trade_amount': 150,
    'atr_multiplier_tp': 10.0,   
    'atr_multiplier_sl': 3.0,  
    'initial_trading_cash': 1000,
    'verbose': False
}

#### Backtest solo DCA

In [81]:
resultados_dca, df_dca = backtest_dca_pure(df, monthly_invest=400)

 Resultados para la estrategia: DCA Puro 
Valor Final del Portafolio: $188,975.88
Total Aportado: $67,200.00
Retorno Porcentual: 181.21%



#### Backtest DCA + Señales de compra con StopLoss SIN ATR

In [82]:
print(" Iniciando Evaluación con Stops sin ATR")
df_resultados_finales = evaluate_all_strategies(df, estrategias_a_probar, backtest_dca_plus_trading_WITH_SL,params)

print("\n Tabla Comparativa de Estrategias")
display(df_resultados_finales)

 Iniciando Evaluación con Stops sin ATR
Evaluando: signal_ema_price...
Evaluando: signal_macd_buy...
Evaluando: signal_stochastic_buy...
Evaluando: signal_ichimoku_kijun_cross...
Evaluando: signal_bollinger_buy...
Evaluando: signal_macd_&_stochastic...
Evaluando: signal_ema_price_&_macd...
Evaluando: signal_ichimoku_kijun_cross_&_stochastic...
Evaluando: signal_ema_price_&_stochastic...

 Tabla Comparativa de Estrategias


,Strategy,Final Portfolio Value,Total Contributions,Absolute Return,Percentage Return,Trading PnL,Trading Capital Used,Open Trades at End,Value of Open Trades
0,signal_ichimoku_kijun_cross (With SL),190878.354942,68200,122678.354942,179.880286,871.664895,21000,2,330.806686
1,signal_macd_buy (With SL),190213.100376,68200,122013.100376,178.904839,237.217014,4500,0,0.000000
2,signal_ema_price (With SL),190036.657817,68200,121836.657817,178.646126,60.774455,3750,0,0.000000
3,signal_stochastic_buy (With SL),190021.976620,68200,121821.976620,178.624599,46.093258,3150,0,0.000000
4,signal_ema_price_&_macd (With SL),190013.808512,68200,121813.808512,178.612622,37.925150,300,0,0.000000
5,signal_ichimoku_kijun_cross_&_stochastic (With...,190006.816423,68200,121806.816423,178.602370,30.933061,1800,0,0.000000
6,signal_bollinger_buy (With SL),189975.883362,68200,121775.883362,178.557014,0.000000,0,0,0.000000
7,signal_macd_&_stochastic (With SL),189975.883362,68200,121775.883362,178.557014,0.000000,0,0,0.000000
8,signal_ema_price_&_stochastic (With SL),189975.883362,68200,121775.883362,178.557014,0.000000,0,0,0.000000


#### Backtest DCA + Señales de compra con StopLoss CON ATR

In [83]:

df_results_atr = evaluate_all_strategies(df, estrategias_a_probar,  backtest_with_atr_SL, params_atr)
print("\n Tabla Comparativa con Stops con ATR ")
display(df_results_atr)

Evaluando: signal_ema_price...
Evaluando: signal_macd_buy...
Evaluando: signal_stochastic_buy...
Evaluando: signal_ichimoku_kijun_cross...
Evaluando: signal_bollinger_buy...
Evaluando: signal_macd_&_stochastic...
Evaluando: signal_ema_price_&_macd...
Evaluando: signal_ichimoku_kijun_cross_&_stochastic...
Evaluando: signal_ema_price_&_stochastic...

 Tabla Comparativa con Stops con ATR 


,Strategy,Final Portfolio Value,Total Contributions,Absolute Return,Percentage Return,Trading PnL,Trading Capital Used,Open Trades at End,Value of Open Trades
0,signal_ichimoku_kijun_cross (ATR Stops),190767.966015,68200,122567.966015,179.718425,748.454803,20850,3,493.62785
1,signal_ema_price (ATR Stops),190235.884859,68200,122035.884859,178.938248,260.001497,3750,0,0.00000
2,signal_macd_buy (ATR Stops),190157.310683,68200,121957.310683,178.823036,181.427321,4500,0,0.00000
3,signal_ema_price_&_macd (ATR Stops),190009.787989,68200,121809.787989,178.606727,33.904627,300,0,0.00000
4,signal_stochastic_buy (ATR Stops),190006.989649,68200,121806.989649,178.602624,31.106287,3150,0,0.00000
5,signal_bollinger_buy (ATR Stops),189975.883362,68200,121775.883362,178.557014,0.000000,0,0,0.00000
6,signal_macd_&_stochastic (ATR Stops),189975.883362,68200,121775.883362,178.557014,0.000000,0,0,0.00000
7,signal_ema_price_&_stochastic (ATR Stops),189975.883362,68200,121775.883362,178.557014,0.000000,0,0,0.00000
8,signal_ichimoku_kijun_cross_&_stochastic (ATR ...,189934.903601,68200,121734.903601,178.496926,-40.979761,1800,0,0.00000


#### Backtest DCA + Señales de compra sin StopLoss ni ATR

In [84]:

df_resultados_finales = evaluate_all_strategies(df, estrategias_a_probar, backtest_dca_plus_trading_NO_SL,params)


print("\n Tabla Comparativa de Estrategias sin Stop ")
display(df_resultados_finales)

Evaluando: signal_ema_price...
Evaluando: signal_macd_buy...
Evaluando: signal_stochastic_buy...
Evaluando: signal_ichimoku_kijun_cross...
Evaluando: signal_bollinger_buy...
Evaluando: signal_macd_&_stochastic...
Evaluando: signal_ema_price_&_macd...
Evaluando: signal_ichimoku_kijun_cross_&_stochastic...
Evaluando: signal_ema_price_&_stochastic...

 Tabla Comparativa de Estrategias sin Stop 


,Strategy,Final Portfolio Value,Total Contributions,Absolute Return,Percentage Return,Trading PnL,Trading Capital Used,Open Trades at End,Value of Open Trades
0,signal_ichimoku_kijun_cross (No SL),192076.551165,68200,123876.551165,181.637172,2044.183758,17100,8,1256.484045
1,signal_macd_buy (No SL),190473.557056,68200,122273.557056,179.286741,499.262791,4050,1,148.410903
2,signal_ema_price (No SL),190437.663219,68200,122237.663219,179.234110,457.742491,3750,1,154.037366
3,signal_stochastic_buy (No SL),190378.685379,68200,122178.685379,179.147633,402.802017,3150,0,0.000000
4,signal_ichimoku_kijun_cross_&_stochastic (No SL),190208.502058,68200,122008.502058,178.898097,232.618696,1800,0,0.000000
5,signal_ema_price_&_macd (No SL),190013.808512,68200,121813.808512,178.612622,37.925150,300,0,0.000000
6,signal_bollinger_buy (No SL),189975.883362,68200,121775.883362,178.557014,0.000000,0,0,0.000000
7,signal_macd_&_stochastic (No SL),189975.883362,68200,121775.883362,178.557014,0.000000,0,0,0.000000
8,signal_ema_price_&_stochastic (No SL),189975.883362,68200,121775.883362,178.557014,0.000000,0,0,0.000000



### Hipótesis 1: Sobre la Viabilidad de la Inversión Activa VS Pasiva


**Hipótesis Nula (H₀)**: Una estrategia de inversión activa, que combina DCA con trading, no genera un rendimiento superior al de una estrategia puramente pasiva de aportaciones periódicas (DCA Puro).

**Hipótesis Alternativa (H₁)**: Una estrategia de inversión activa bien diseñada sí puede generar un rendimiento superior al de la estrategia pasiva, superando el "coste de oportunidad".

- Generando una estrategia de solo DCA (inversion pasiva, comprar y mantener) se genera un retorno del 181,20% mientras que con una estrategia activa (Combinacion de DCA con trades) se genera 181.63% ligeramente superior. 

### Hipótesis 2: Sobre la Efectividad de los Tipos de Indicadores 

Hipótesis Nula (H₀): No existe una diferencia significativa en el rendimiento entre las estrategias basadas en seguimiento de tendencia (MACD), las de reversión a la media (Estocástico) o las de equilibrio (Ichimoku).

Hipótesis Alternativa (H₁): Para un activo con fuerte tendencia como VOO, las estrategias basadas en equilibrio y tendencia son significativamente más rentables que las de reversión.

- Utilizando estrategias basadas en compra activa segun las señales de indicadores tecnicos, si exixte unas diferences significativas entre las de reversion y las de equilibrio y tendencia.
- Utilizando Ichimoku vemos como el capital que se invierte en un primer momento (capital inicial = 1000$) al final del ciclo es recuperado ademas de doblado y representando un 12% del capital reinvertido en el periodo estudiado. 
    

### Hipótesis 3: Sobre la Gestión del Riesgo (El Impacto del Stop-Loss) 

Esta hipótesis se centra en cómo la gestión de riesgo afecta los resultados en un mercado específico.

Hipótesis Nula (H₀): La implementación de un stop-loss mejora o no afecta negativamente el rendimiento total al limitar las pérdidas.

Hipótesis Alternativa (H₁): En un mercado fuertemente alcista, la eliminación del stop-loss mejora el rendimiento total, ya que evita la venta prematura en caídas temporales.

- la estrategia con mejor PNL y mejores resultados absolutos es aquella en la que utilizamos ichimoku sin el stop loss lo que hace que se rechaze la hipotesis nula de que el stop loss mejora el rendimiento. 

In [85]:
def crear_target(df, dias_futuro=20, umbral_retorno=0.08):
    """
    Crea la variable objetivo 'target'.
    Será 1 si el precio sube más del 'umbral_retorno' en los próximos 'dias_futuro'.
    """
    # shift(-dias_futuro) trae el precio de cierre de dentro de 20 días a la fila actual
    df['retorno_futuro'] = df['close'].shift(-dias_futuro) / df['close'] - 1
    
    # Crea el target: 1 si el retorno supera el umbral, 0 si no
    df['target'] = (df['retorno_futuro'] > umbral_retorno).astype(int)
    
    # Limpiamos las últimas filas que tendrán NaNs por el shift
    df = df.dropna(subset=['retorno_futuro', 'target'])
    return df



In [86]:
# Aplicar la función para crear un target viable: aqui dejamos un target en el que le indicamos si el precio sube un 4% en 19 dias. 
df = crear_target(df, dias_futuro=19, umbral_retorno=0.04)

In [87]:
df.head()

,date,close,high,low,open,volume,daily_return,rsi,macd,macd_signal,...,signal_ichimoku_aggressive,signal_ichimoku_kijun_cross,signal_ichimoku_chikou,signal_bollinger_buy,signal_macd_&_stochastic,signal_ema_price_&_macd,signal_ichimoku_kijun_cross_&_stochastic,signal_ema_price_&_stochastic,retorno_futuro,target
0,2011-06-23,91.127,91.158,89.689,90.338,260750,-0.003,44.614,-0.773,-0.887,...,False,False,False,False,False,False,False,False,0.048679,1
1,2011-06-24,90.125,91.181,89.969,91.150,114500,-0.011,39.853,-0.802,-0.870,...,False,False,False,False,False,False,False,False,0.061204,1
2,2011-06-27,90.886,91.228,89.985,90.187,77850,0.008,44.681,-0.756,-0.847,...,False,False,False,False,False,False,False,False,0.046333,1
3,2011-06-28,92.051,92.082,91.181,91.274,59500,0.013,51.147,-0.618,-0.801,...,False,False,False,False,False,False,False,False,0.028701,0
4,2011-06-29,92.875,92.999,92.191,92.533,125350,0.009,55.141,-0.436,-0.728,...,False,True,False,False,False,False,False,False,-0.000668,0


In [ ]:
# Creación de Características Numéricas Avanzadas 

# a) Características de las Velas (Candlestick Features)
df['body_size'] = abs(df['close'] - df['open'])
df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)
df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']

# b) Momentum de los Indicadores (Rate of Change)
df['rsi_roc_5'] = df['rsi'].pct_change(periods=5) * 100

# c) Características Cíclicas del Tiempo
df['month_sin'] = np.sin(2 * np.pi * df['date'].dt.month / 12)
df['month_cos'] = np.cos(2 * np.pi * df['date'].dt.month / 12)

# d)creamos cruzes del precio con kijun y los cruces del tenkan y kijun
df['price_vs_kijun'] = df['close'] - df['ichimoku_base']
df['tenkan_vs_kijun'] = df['ichimoku_conversion'] - df['ichimoku_base']

# Limpiar los NaNs generados por los nuevos cálculos
df = df.dropna()



In [89]:

#exportamos los datos a un csv en la carpeta data
df.to_csv('../data/VOO_ind_signal.csv', index=False)